# Imports


In [77]:
import numpy as np
import os
import matplotlib.pyplot as plt

# Access functions

In [78]:
def retrieve_results(path):
    """
    This function retrieves the results of a certain computation.

    Input:
     - path: inner path to the folder containing a result (consider to address a path/job.out)

    Output:
     - A numpy array of 3 unsigned integers (matched patterns and the 2 checksums),
       or
     - [-1, -1, -1] meaning an absence of attemps
       or
     - [-2, -2, -2] meaning a non-finished run due to interruption
       or
     - [-3, -3, -3] meaning an aborted run
    """
    # Checking that the computation was at least attempted
    if "job.out" and "job.err" in os.listdir(path):
        f = open(path + "/job.err", "r")
        # Checking that the computation had no errors
        if (len(f.readlines()) > 0):
            return np.array([-3, -3, -3])

        f = open(path + "/job.out", "r")

        # Checking that the computation has gone through correctly
        lines = f.readlines()
        if len(lines) > 0:
            result = lines[2].split()[1:]
            result[0] = result[0][:-1]
            result[1] = result[1][:-1]
            return np.array(result, dtype=np.uint64)
        else:
            return np.array([-2, -2, -2])
        
    return np.array([-1, -1, -1])


In [79]:
# Test

retrieve_results("results/logs/sequential/seq_length_25/patterns_15/mean_path_length_4")

array([-2, -2, -2])

In [80]:
def retrieve_settings(path):
    """
    This function retrieves all the settings for a certain measurement.

    Input:
     - path: inner path to the folder containing a result (consider to address a path/job.out)

    Output: 
     - A numpy array of settings in format np.int64
    """
    options = path.split("/")
    settings = np.array([], dtype=np.int64)
    for set in options:
        if not str.isalpha(set.split("_")[-1]):
            settings = np.append(settings, np.int64(set.split("_")[-1]))
            
    return settings

In [81]:
# Test

retrieve_settings("results/logs/sequential/seq_length_25/patterns_15/mean_path_length_4")
[os.path.isdir("results/logs/sequential" + "/" + x) for x in os.listdir("results/logs/sequential")]

[True, False, True, True, True]

In [82]:
# These indexes allow us to rapidly swap the number of parameter in consideration during generation.

sequential_idx = {"seq_length": 0, "patterns": 1, "mean_path_length": 2}
pthreads_idx = {"threads": 0, "seq_length": 1, "patterns": 2, "mean_path_length": 3}

# Dataset creation

In [83]:
def create_dataset(path, dataset):
    """
    This function recursively explores the results folder and gathers data about the measurements.

    Input:
     - path: the path of the dataset
     - dataset: whatever numpy array of np.uint with the same dimension as the measurement variables

     Output: 
      - The dataset as a numpy array of np.uint. The shape of the array is (measurements, variables),
        where the last 3 variables are the results of the computation
    """
    new_dataset = np.copy(dataset)
    if not os.path.isdir(path):
        return dataset
    elif not np.any([os.path.isdir(path + "/" + x) for x in os.listdir(path)]):
        results = retrieve_results(path)
        settings = retrieve_settings(path)
        row = np.append(settings, results.astype(np.int64)).reshape(1, -1)
        return np.append(new_dataset, row, axis=0)
    else:
        for dir in os.listdir(path):
            new_dataset = create_dataset(path + "/" + dir, new_dataset)

    return new_dataset

In [84]:
# Testing

sequential = create_dataset("results/logs/sequential", np.zeros(shape=(1, len(sequential_idx) + 3), dtype=np.int64))[1:,]

## Data retrieval


In [92]:
pthreads = create_dataset("results/logs/pthreads", np.zeros(shape=(1, len(pthreads_idx) + 3), dtype=np.int64))[1:,]
sequential = create_dataset("results/logs/sequential", np.zeros(shape=(1, len(sequential_idx) + 3), dtype=np.int64))[1:, :]

In [93]:
sequential[np.lexsort(np.transpose(sequential)[::-1])]

array([[   10,    10,     4,  1147,   368, 39054],
       [   10,    10,     6,  1024, 13697, 64955],
       [   10,    10,     8,  1024, 52130, 65257],
       [   10,    10,    10,    -1,    -1,    -1],
       [   10,    15,     4, 36701, 13109, 31218],
       [   10,    15,     6, 32772, 64241, 45148],
       [   10,    15,     8, 32768, 49010, 51114],
       [   10,    15,    10,    -1,    -1,    -1],
       [   10,    20,     4,    -1,    -1,    -1],
       [   10,    20,     6,    -1,    -1,    -1],
       [   10,    20,     8,    -1,    -1,    -1],
       [   10,    20,    10,    -1,    -1,    -1],
       [   10,    25,     4,    -1,    -1,    -1],
       [   10,    25,     6,    -1,    -1,    -1],
       [   10,    25,     8,    -1,    -1,    -1],
       [   10,    25,    10,    -1,    -1,    -1],
       [   25,    10,     4,  1323, 28815, 23226],
       [   25,    10,     6,  1024, 14516, 65439],
       [   25,    10,     8,  1024, 15615, 65006],
       [   25,    10,    10,   

# Checking correctness of computation

In [101]:
def correctness(path, method_idx):
    """
    This function checks the correctness of a computation and tests it against sequential's results.

    Input:
     - path: path to the results
    
    Output:
     - self-described
    """
    # Creating the results datasets
    sequential = create_dataset("results/logs/sequential", np.zeros(shape=(1, len(sequential_idx) + 3), dtype=np.int64))[1:,]
    sequential = sequential[np.lexsort(np.transpose(sequential)[::-1])]
    dataset = create_dataset(path, np.zeros(shape=(1, len(method_idx) + 3), dtype=np.int64))[1:,]
    dataset = dataset[np.lexsort(np.transpose(dataset)[::-1])]

    # First check: runs with errors
    errors = dataset[np.where(dataset[:, -3:] == np.array([-3, -3, -3], dtype=np.int64))]
    if errors.shape[0] == 0:
        print("1. No errors")
    else:
        print(f"1. There where {errors.shape[0]} errors:")
        print(errors)

    # Second checks: runs non - terminated (where sequential terminated)
    errors = sequential[(np.where(sequential[:, -3:] == np.array([-2, -2, -2], dtype=np.int64))) &
                        (np.where(sequential[:, :-3] == np.array([-2, -2, -2], dtype=np.int64)))]
    if errors.shape[0] == 0:
        print("1. No errors")
    else:
        print(f"1. There where {errors.shape[0]} errors:")
        print(errors)

In [102]:
correctness("results/logs/pthreads", pthreads_idx)

1. No errors
